# Import Dependencies

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
import numpy as np
import cv2
import imutils

from ChannelFeatures import ChannelFeatures
from feature_generator import FeatureGenerator
import nms

# Detector Class

In [6]:
class Detector():
    """ 
        The Detector class is used to detect pedestrians in images by locating bounding boxes with high probabilities
        containing a pedestrian 
    """

    def __init__(self, clf, fg, window_size=(120,60), scaling_factor=1.2, scaling_iters=1, window_step=6):
        """ Instantiates the detector class:
        
            Input: weight_indices, weights, window_size, scaling_factor, scaling_iters, window_step
            
            - weight_indices: the indices of the features that will be used to score a window in the image
            - weights: the weights used to compute a score for a feature vector associated with a window in the image
            - fg: FeatureGenerator() object used to generate feature vectors for window in an image
            - window_size: the size of the sliding window defaults to 120 x 60
            - scaling_factor: factor by which we scale the image on each successive scaling iteration
            - scaling_iters: the number of times we scale the image
            - window_step: the amount of pixels stepped over on each slide of the window
        """

        self.clf = clf
        self.window_size = window_size
        self.window_step = window_step

        self.scaling_factor = scaling_factor
        self.scaling_iters = scaling_iters

        self.cf = ChannelFeatures()
        self.fg = fg

    def detect_pedestrians(self, img_path):
        """
            Detects pedestrians in an image.

            1) Slides bounding box window over the image
            2) Computes detection score using weights from boosted tree classifier
            3) Keeps the bounding box if the score is above a certain threshold
            4) Runs non-maximal suppression (NMS) on bounding boxes

            Input: img_path

            - img_path: path to image file

            Output: list of bounding boxes and scores 

        """
        
        candidate_bbs = self._get_bounding_boxes(img_path)
        bbs = nms.non_max_suppression(np.asarray(candidate_bbs), overlapThresh=0.5)
        return candidate_bbs, bbs
        

       
        
    def _get_bounding_boxes(self, img_path, start_h=120, start_w=60):
        """ 
            Returns 2D array of bounding boxes (M bounding boxes x 5 characteristics per bounding box)
        """
        
        bounding_boxes = []
    
        
        img = cv2.imread(img_path)
        oheight, owidth, channels = img.shape

        if oheight/start_h > owidth/start_w:
            img = imutils.resize(img, width=min(start_w,img.shape[1]))
        else:
            img = imutils.resize(img, height=min(start_h,img.shape[0]))
            
        cv2.imwrite('resized_img.jpeg',img)
        
        oheight, owidth, channels = img.shape
        win_h, win_w = self.window_size

        count = 0
        print self.scaling_iters
        
        #=====[ Collect bounding boxes for each scaling iteration ]=====
        for it_num in range(self.scaling_iters):

            #=====[ Scale image if not on first iteration ]=====
            if it_num > 0:
                img = cv2.resize(img,(int(it_num*self.scaling_factor*owidth), int(it_num*self.scaling_factor*oheight)))

            height, width, _ = img.shape

            y_range = (height - win_h)/self.window_step + 1
            x_range = (width - win_w)/self.window_step + 1
            
            cfeats = self.cf.compute_channels(img)
            
            print "height:", height, "    width:", width
            print y_range, x_range

            #=====[ Slide window across entirety of image and calculate bounding box at each step ]=====
            for y in range(y_range):
                for x in range(x_range):

                    y_pix = y*self.window_step
                    x_pix = x*self.window_step

                    #=====[ Score the bounding box ]=====
                    feature_vec = np.asarray(self.fg.generate_features(cfeats[y:y+win_h,x:x+win_w]))
                    
                    score = self.clf.predict_proba([feature_vec])[0,1]

                    #=====[ Scale and store bounding box ]=====
                    scale = self.scaling_factor*it_num if it_num else 1
                    count += 1
                    
                    if score > 0.5:
                        bounding_boxes.append([score, y_pix/scale, x_pix/scale, win_h/scale, win_w/scale])


            print 'Went through %d total candidate BBs' %(count)
        return np.matrix(bounding_boxes)
        
    def _calculate_total_iters(self, img):
        """ Calculates total number of bounding box scores to be calculated """
        
        oheight, owidth, channels = img.shape
        win_h, win_w = self.window_size
        
        iters = 0
        
        for it_num in range(self.scaling_iters):

            #=====[ Scale image if not on first iteration ]=====
            if it_num > 0:
                img = cv2.resize(img,(int(it_num*self.scaling_factor*owidth), int(it_num*self.scaling_factor*oheight)))

            height, width, _ = img.shape

            y_range = (height - win_h)/self.window_step + 1
            x_range = (width - win_w)/self.window_step + 1
            
            iters += y_range*x_range
        
        return iters

In [3]:
from template_generator import TemplateGenerator
import pickle
import sys
sys.path.append('../')
from detector import Detector

print "BB Detection for full feature set classifier is about 7 minutes"
print "BB Detection for small feature set classifier is about 2 minutes"

clf = pickle.load(open('../BoostedTreeclassifier.p','rb'))
print "%d Estimators Used" % (len(clf.clf.estimator_weights_))
print "%d Features" % (len(clf.clf.feature_importances_))

tg = TemplateGenerator()
# tg.generate_sizes(w_max=3, h_max=2)
tg.generate_sizes()
templates = tg.generate_templates()

#=====[ Instantiate FeatureGenerator ]=====
fg = FeatureGenerator(templates)

d = Detector(clf.clf,fg)

BB Detection for full feature set classifier is about 7 minutes
BB Detection for small feature set classifier is about 2 minutes
2000 Estimators Used
24926 Features
Created 2266 templates


In [4]:
import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

print datetime.datetime.now()
cand_bbs, bbs = d.detect_pedestrians('../ped_image.jpeg')
print datetime.datetime.now()

2016-06-01 00:35:43.571376
1
height: 120     width: 160
1 17
Went through 17 total candidate BBs
2016-06-01 00:35:49.073677


In [5]:
print '%d Bounding boxes extracted' % (len(bbs))

1 Bounding boxes extracted


In [269]:
bb = np.asarray(bb)

nms_bb = nms.non_max_suppression(bb, overlapThresh=0.1)
print '%d Bounding Boxes after suppression' % (len(nms_bb))


img = cv2.imread('resized_img.jpeg')
for box in nms_bb:
    print 'Score is :', box[0]
    cv2.rectangle(img,(int(box[2]),int(box[1])),(int(box[2]+box[4]),int(box[1]+box[3])),(0,255,0),3)    

cv2.imwrite('saved_img.jpeg',img)

5 Bounding Boxes after suppression
Score is : 0.547428661207
Score is : 0.542531193857
Score is : 0.524270617203
Score is : 0.51161661939
Score is : 0.503109226978


True